In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import pandas as pd

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('smma_p', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))
            
            
            
    

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders, buy price/commission, and ntrades
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.ntrades = 0

        # Smotthed Moving Average + MACD
        self.smma = bt.indicators.SMMA(
            self.datas[0], period=self.params.smma_p)
        self.macd = bt.indicators.MACD(
            self.datas[0],
            period_me1=3,
            period_me2=10,
            period_signal=6)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
                
            self.bar_executed = len(self)
            self.ntrades += 1


        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.ntrades += 1
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if (self.smma[0] < self.dataclose[0]) and \
               (self.macd.macd[0] > 0) and \
               ((self.macd.signal[0] > 0) and (self.macd.macd[0] > self.macd.signal[0])):


                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.smma[0] > self.dataclose[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(SMMA Period %2d) Ending Value %.2f Trades %2d' %
                 (self.params.smma_p, self.broker.getvalue(), self.ntrades), doprint=True)

In [ ]:
# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-01-01'])

## In-sample Optimization

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
strats = cerebro.optstrategy(
    TestStrategy,
    smma_p=range(10, 40),
)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Set the commission
cerebro.broker.setcommission(commission=0.0)
cerebro.broker.set_slippage_fixed(.01, 
                              slip_open=True, 
                              slip_limit=True, 
                              slip_match=True, 
                              slip_out=True)
# Run over everything
results = cerebro.run(maxcpus=1)

## In-Sample Best

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy,
                    smma_p=12
                   )

# Add the Data Feed to Cerebro
cerebro.adddata(data)

cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Months)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')



# Set our desired cash start
cerebro.broker.setcash(100000.0)

cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.broker.setcommission(commission=0.0)
cerebro.broker.set_slippage_fixed(.01,
                              slip_open=True, 
                              slip_limit=True, 
                              slip_match=True, 
                              slip_out=True)
# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
in_result = cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(volume=False, iplot=False)
in_sample_sharpe = in_result[0].analyzers.sharpe.get_analysis()
in_sample_drawdown = in_result[0].analyzers.drawdown.get_analysis()

## Out-of-Sample

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy,
                    smma_p=12,
                   )


out_data = bt.feeds.PandasData(dataname = df['2018-01-01':'2018-06-01'])


# Add the Data Feed to Cerebro
cerebro.adddata(out_data)

cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Months)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')


# Set our desired cash start6
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.broker.setcommission(commission=0.0)
cerebro.broker.set_slippage_fixed(.01, 
                              slip_open=True, 
                              slip_limit=True, 
                              slip_match=True, 
                              slip_out=True)
# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
out_result = cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(volume=False, iplot=False)
out_sample_sharpe = out_result[0].analyzers.sharpe.get_analysis()
out_sample_drawdown = out_result[0].analyzers.drawdown.get_analysis()

In [ ]:
print('in sample:')
print(in_sample_sharpe)

print('out sample:')
print(out_sample_sharpe)
